In [ ]:
# !wget https://huggingface.co/datasets/mesolitica/mixtral-magicoder/resolve/main/data/python-00000-of-00001.jsonl
# !wget https://huggingface.co/datasets/mesolitica/mixtral-magicoder/resolve/main/data/pythonanalytics-00000-of-00001.jsonl

In [ ]:
# !wget https://huggingface.co/datasets/m-a-p/Code-Feedback/resolve/main/Code-Feedback.jsonl
# !wget https://huggingface.co/datasets/teknium/OpenHermes-2.5/resolve/main/openhermes2_5.json

In [ ]:
import json

with open('prepared-openhermes.json') as fopen:
    filtered = json.load(fopen)
    
filtered = [r['conversations'][0]['value'] for r in filtered]
filtered = [r for r in filtered if len(r) > 10]

In [ ]:
len(filtered)

In [ ]:
!mkdir openhermes

In [ ]:
import docker

client = docker.from_env()
for c in client.containers.list(filters = {'name': f'conv-openhermes-*'}):
    try:
        c.remove(force = True)
    except:
        pass
client.close()

In [ ]:
from agent import ClientJupyterKernel, Agent, Generator
import os
import docker

def answer(q, i):
    filename = f'openhermes/{i}.json'
    if os.path.exists(filename):
        return
    
    jupyter_kernel_url = 'http://localhost:8081/execute'
    openai_base_url = 'http://localhost:8005/v1'
    model_name = 'xingyaoww/CodeActAgent-Mistral-7b-v0.1'

    generator = Generator(model_name, openai_base_url)
    prompt = q
    
    CONV_ID = None
    
    r = None
    agent = None
    for k in range(2):
        try:
            CONV_ID = f'openhermes-{i}-{k}'
            code_executor = ClientJupyterKernel(jupyter_kernel_url, CONV_ID)
            agent = Agent(generator, code_executor, conv_id=CONV_ID, silent = True)
            r = agent.run_prompt(prompt, n_max_executions = 10)
            agent.code_executor.shutdown()
            if '<execute>' in r[2]['content']:
                break
        except Exception as e:
            print(e)
            if agent is not None:
                agent.code_executor.shutdown()
            pass
        
    with open(filename, 'w') as fopen:
        json.dump({'original': q, 'result': r}, fopen)
    
    client = docker.from_env()
    for c in client.containers.list(filters = {'name': f'conv-openhermes-{i}-*'}):
        try:
            c.remove(force = True)
        except:
            pass
    client.close()

In [ ]:
def consumer(queue, name):
    while True:
        if queue.qsize() == 0:
            break
        item = queue.get()
        answer(*item)
    print(f'consumer {name} done')

In [ ]:
urls = [(q, no) for no, q in enumerate(filtered)]

In [ ]:
# !rm code-feedback/210.json
answer(*urls[10003])

In [ ]:
# !rm code-feedback/*.json

In [ ]:
# with open('code-feedback/78.json') as fopen:
#     for r in json.load(fopen)['result']:
#         print(r)

In [ ]:
from threading import Thread
from queue import Queue
from tqdm import tqdm

queue = Queue()
for u in urls:
    queue.put(u)
    
ori_size = queue.qsize()

In [ ]:
max_worker = 50
consumers = [Thread(target=consumer, args=(queue,i)) for i in range(max_worker)]
for i in range(len(consumers)):
    consumers[i].start()
    
pbar = tqdm(total=ori_size)
last_size = 0
while True:
    size = queue.qsize()
    if size == 0:
        break
    left = ori_size - size
    minus = left - last_size
    if minus > 0:
        pbar.update(minus)
        last_size += minus

pbar.close()